# Dependencies


In [ ]:
# Pytube
!pip install pytube

# Whisper
!pip install git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2lcwli4x
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2lcwli4x
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manyli

# Imports

In [ ]:
from pytube import YouTube
import whisper
import google.generativeai as genai
import json

# Download YT Quiz video's audio

In [ ]:
def download_yt_audio(url, output_path='./'):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    output_file = video.download(output_path=output_path)
    return output_file

In [ ]:
audio_file = download_yt_audio('https://www.youtube.com/watch?v=b3B2gu_tIRI')
print(audio_file)

/content/./Can Europeans Answer Simple Geography Questions.mp4


# Transcribe the audio

In [ ]:
def transcribe(audio_file):

  model = whisper.load_model("small")

  transcript = model.transcribe(audio_file)
  return transcript

In [ ]:
transcript = transcribe(audio_file)
print(transcript["text"])

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 70.0MiB/s]


 Plus minus three, how many countries worldwide speak Spanish as their official language? Um, I think 21. That is precisely accurate, it's 20. Holy s***. What is the zero degrees latitude line called? I don't know. It will be equator. Oh my gosh, did I really say that? Oh, equator. Today I'm here in Bucharest, Romania, attending the EGM, the Erasmus Generation Meeting, where more than 800 people from all around Europe who are part of the Erasmus Student Network gather to have discussions, workshops, sessions in order to shape the future of Europe. For every question that they answer correctly, they win 20 euros. Let's see who's gonna win some money. What's your name? Eiberk. Where are you from? Turkey. Name three countries that have the same four last letters. Spain. The Netherlands. No, no, no. Same that end with the same four last letters. Bulgaria. Slovakia. Slovenia. No, it's A. No, no, no. With the same four last letters. Oh my god. Uzbekistan, Tajikistan, Kyrgyzstan. There you go

# Extract the Questions and Answers

In [ ]:
def build_prompt(transcript):
  return """ Below is a transcript with a person quizzing other people. Read
  the transcript and extract all the questions and their corresponding answers
   and put them in the format:
   [
      {
        "question": "Question",
        "answer": "Answer"
      }
   ]
   Here is the transcript: \n
  """ + transcript

def get_quiz_questions_and_answers(transcript):
  prompt = build_prompt(transcript)

  genai.configure(api_key="YOUR_API_KEY")
  text_model = genai.GenerativeModel('gemini-pro')
  prompt = build_prompt(transcript)
  text_response = text_model.generate_content(prompt)
  return text_response.text



In [ ]:
quiz = get_quiz_questions_and_answers(transcript["text"])
print(quiz)

[
  {
    "question": "Plus minus three, how many countries worldwide speak Spanish as their official language?",
    "answer": "20"
  },
  {
    "question": "What is the zero degrees latitude line called?",
    "answer": "Equator"
  },
  {
    "question": "Name three countries that have the same four last letters.",
    "answer": "Bulgaria, Slovakia, Slovenia"
  },
  {
    "question": "Name four countries that end with Stan.",
    "answer": "Uzbekistan, Tajikistan, Kyrgyzstan"
  },
  {
    "question": "Name four capitals worldwide that have four letters.",
    "answer": "Delhi, Rome, Lima, Doha"
  },
  {
    "question": "On which continent is the youngest country in the world?",
    "answer": "Africa"
  },
  {
    "question": "Which country is the youngest in the world?",
    "answer": "South Sudan"
  },
  {
    "question": "Name three out of four capitals of the UK.",
    "answer": "London, Edinburgh, Cardiff"
  },
  {
    "question": "Which continent is the driest one in the world?"

# Save the the json string in a file to upload in the Android App

In [ ]:
with open('europeans_geography_questions_quiz.json', 'w') as outfile:
    json.dump(quiz, outfile)